<a href="https://colab.research.google.com/github/zzl120/mnist_tutorial/blob/master/stylegan2_ada_pytorch_pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Pokemon with StyleGAN

In this tutorial, we only want to familiarize ourselves with running StyleGAN (in this case, the newest version) and seeing how training progresses.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [39]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-45037350-b07e-1a7f-3861-c9c38e64277b)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install Stylegan2-ada-pytorch Prerequisites

In [41]:
import torch

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

In [42]:
torch.cuda.device_count()

1

In [43]:
import torchvision

In [44]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

## Get the StyleGAN code

In [47]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.


In [48]:
mkdir /content/stylegan2-ada-pytorch/datasets

mkdir: cannot create directory ‘/content/stylegan2-ada-pytorch/datasets’: File exists


## Get Pokemon dataset of images

In [49]:
cd /content/stylegan2-ada-pytorch/datasets

/content/stylegan2-ada-pytorch/datasets


In [51]:
cd custom

/content/stylegan2-ada-pytorch/datasets/custom


In [52]:
ls

00013edfe31d2ce237ee1c6df676e5a3.png  7dbab035c85c2bf8823b88b7f2fd4a5f.png
0012b9a4cc7fac34bd2c3ea9f93961a4.png  7dda34887f978f7e9c7f325acfff4057.png
002a4dfc2199b01eabb1ffcb58f1ed29.png  7e0ae1403d78c09348b61402ef2046ba.png
002d853a7272ce55fb368c69757a3acc.png  7e388e7b042974f79a7b4c2d8e7d7491.png
005377f5f29e5585fe5e9cc0788a0ae9.png  7e702791e2db1a687f5cb557b7dbad72.png
008d12c81e507113107b1908c09bb9e9.png  7e82d77051df22a7a21e8743b48a64e2.png
00db48bce048863477b0d4e88e099ecb.png  7e86869e2e76ba5bce62b4d784723005.png
010354469223b993f1d0dbf2a78d765a.png  7e92362f8c01c1c21b0663890209e59a.png
010a48affaa193411134fa76eaa2edfe.png  7ea05ce82056e76f5700479d792969b3.png
012f4c8e0520c0b712404d7d2b10fde2.png  7ecb145154dcacaabf2f1eca399aef78.png
0131becdd42eba726f6f85e67cc42008.png  7f2514baf6503a5e5a09342898fa905c.png
013f2fd09843d0a0623ba7d527b56f54.png  7f25e1feda4150a933d9ba63efc01a20.png
01515053ab90bee825b4cd719b5825af.png  7f45a020e3a61e55c8ab9ef626c025f4.png
0163735960550dd9ba05bc73a

In [53]:
cd ..

/content/stylegan2-ada-pytorch/datasets


In [ ]:
!rm pokemon256.zip

In [54]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


## Prepare Pokemon dataset for use by StyleGAN

In [55]:
!python dataset_tool.py --source=./datasets/custom --dest=./datasets/custom.zip

100% 2011/2011 [00:08<00:00, 225.42it/s]


## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  The Pokemon training data will be stored in **MachineLearningForArtists/Pokemon**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/Pokemon

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/Pokemon --data=./datasets/pokemondataset.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=10 --metrics=none


## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/Pokemon

Each training run is stored in a separate directory.  The initial training run is stored in `00000-pokemondataset...`.  If you run training twice, the second run will be stored in `00001-pokemondataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various Pokemon in this image. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Transfer Learning

Now, let's train the same dataset but use *transfer learning*.  We will use a StyleGAN2 model already pre-trained on Pokemon to train a new model that is trained against a Tamagotchi dataset.

### Get the Tomagotchi dataset

In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets

In [ ]:
!wget https://github.com/derekphilipau/machinelearningforartists/raw/main/docs/src/week2/Tamagotchi256.zip

In [ ]:
!unzip Tamagotchi256.zip

In [ ]:
!rm -rf __MACOSX/

In [ ]:
%cd /content/stylegan2-ada-pytorch

### Prepare Tomagotchi dataset for use by StyleGAN

In [ ]:
!python dataset_tool.py --source=./datasets/Tamagotchi256 --dest=./datasets/tamagotchidataset.zip

### Get the pre-trained Pokemon model

In [56]:
!gdown --id 1wFyoYM5pji_H6g-OyvxOTX80L6PkBA4E

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1wFyoYM5pji_H6g-OyvxOTX80L6PkBA4E 



### Train using the Pokemon model with the Tamagotchi dataset 

In [ ]:
!python train.py --resume=./stylegan2-ada-pytorch-pokemon-256px-11650.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/Tamagotchi --data=./datasets/tamagotchidataset.zip --gpus=1 --augpipe=bg --gamma=10 --cfg=paper256 --mirror=1 --snap=10 --metrics=none
